# Cricket Ball Detection – Prediction Notebook

Imports OpenCV, NumPy, and supporting libraries.

In [ ]:
from ultralytics import YOLO
import cv2
import csv
import numpy as np


Performs a key step in the video prediction pipeline- a simple Tracker class

In [ ]:
class Tracker:
    def __init__(self):
        self.points = []   # list of (x, y) tuples

    def update(self, x, y):
        self.points.append((x, y))
        return self.points


Loads a video stream and processes frames.

In [ ]:
def process_video(video_path, output_video_path, output_csv_path,
                  model_path, conf_threshold=0.05, imgsz=1280):

    # Load YOLO model
    model = YOLO(model_path)

    # Open video
    cap = cv2.VideoCapture(video_path)

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out    = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Prepare CSV
    csv_file = open(output_csv_path, mode="w", newline="")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["frame", "x", "y"])

    tracker = Tracker()
    frame_idx = 0

    # Loop through frames
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO Inference
        results = model.predict(frame, imgsz=imgsz, conf=conf_threshold, verbose=False)
        detections = results[0].boxes

        ball_found = False
        xc, yc = None, None

        if len(detections) > 0:
            # # Take the top detection
            # box = detections[np.argmax(detections.conf)]
            # Convert confidence tensor -> CPU -> NumPy
            conf = detections.conf.cpu().numpy()

            # Get index of highest-confidence detection
            best_idx = np.argmax(conf)

            # Select that detection
            box = detections[best_idx]


            x1, y1, x2, y2 = box.xyxy[0]

            # Convert to int
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Compute centroid
            xc = int((x1 + x2) / 2)
            yc = int((y1 + y2) / 2)

            ball_found = True

        # Track the point
        if ball_found:
            points = tracker.update(xc, yc)
            csv_writer.writerow([frame_idx, xc, yc])
        else:
            points = tracker.points  # keep old points

        # Draw trajectory
        for i in range(1, len(points)):
            x_prev, y_prev = points[i - 1]
            x_curr, y_curr = points[i]
            cv2.line(frame, (x_prev, y_prev), (x_curr, y_curr), (0, 255, 0), 2)

        # Draw current point
        if ball_found:
            cv2.circle(frame, (xc, yc), 6, (0, 0, 255), -1)

        # Write frame
        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    csv_file.close()

    print("Done! Output saved:")
    print("Video:", output_video_path)
    print("CSV:", output_csv_path)


now use this function to make prediction and process out the tracked videos

In [ ]:
import os
video_folder = "../data (1)/raw_videos (1)/"
output_folder = "all_final_outputs/"

os.makedirs(output_folder, exist_ok=True)

for i in range(1, 15):

    video_path = f"{video_folder}{i} (1).mp4"

    output_video_path = f"{output_folder}{i}_processed.mp4"
    output_csv_path   = f"{output_folder}{i}_trajectory.csv"

    print(f"\n🔹 Processing video {i}: {video_path}")

    process_video(
        video_path=video_path,
        output_video_path=output_video_path,
        output_csv_path=output_csv_path,
        model_path="runs/detect/train6/weights/best.pt",
        conf_threshold=0.05,
        imgsz=1280
    )

print("\n✅ All 15 videos processed successfully!")
